# Spark Apache семинары

## Урок 3. Инструменты работы и визуализации ч.2

### Задача:

Условие: есть набор данных о продажах продуктов с информацией о дате продаж, категории продукта, количестве и выручке от продаж.

Используя Apache Spark, загрузите предоставленный набор данных в DataFrame (пример данных ниже).

("2023-11-20", "Electronics", 100, 12000),

("2023-11-21", "Electronics", 110, 13000),

("2023-11-22", "Electronics", 105, 12500),

("2023-11-20", "Clothing", 300, 15000),

("2023-11-21", "Clothing", 280, 14000),

("2023-11-22", "Clothing", 320, 16000),

("2023-11-20", "Books", 150, 9000),

("2023-11-21", "Books", 200, 12000),

("2023-11-22", "Books", 180, 10000)

Столбцы: "date", "category", "quantity", "revenue".

С использованием оконных функций, рассчитайте среднее выручки от продаж для каждой категории продукта.
Примените операцию `pivot` для того, чтобы преобразовать полученные данные таким образом, чтобы в качестве строк были категории продуктов, в качестве столбцов были дни, а значениями были средние значения выручки от продаж за соответствующий день

In [4]:
import init_spark_env

In [3]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.master("local[*]").appName("My Spark Application").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/02 13:27:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
spark

In [18]:
from pyspark.sql import Window
from pyspark.sql.functions import avg

In [19]:
# Инициализация SparkSession
spark = SparkSession.builder.master("local[*]").appName("Sales Data Analysis").getOrCreate()

In [20]:
# Данные о продажах
sales_data = [
    ("2023-11-20", "Electronics", 100, 12000),
    ("2023-11-21", "Electronics", 110, 13000),
    ("2023-11-22", "Electronics", 105, 12500),
    ("2023-11-20", "Clothing", 300, 15000),
    ("2023-11-21", "Clothing", 280, 14000),
    ("2023-11-22", "Clothing", 320, 16000),
    ("2023-11-20", "Books", 150, 9000),
    ("2023-11-21", "Books", 200, 12000),
    ("2023-11-22", "Books", 180, 10000)
]

# Создание DataFrame с данными о продажах
sales_df = spark.createDataFrame(sales_data, ["date", "category", "quantity", "revenue"])

# Преобразование данных с использованием pivot и вычисление средней выручки
pivot_df = sales_df.groupBy("category") \
                   .pivot("date") \
                   .agg(avg("revenue")) \
                   .orderBy("category")

# Вывод результата
sales_df.show()
pivot_df.show()

+----------+-----------+--------+-------+
|      date|   category|quantity|revenue|
+----------+-----------+--------+-------+
|2023-11-20|Electronics|     100|  12000|
|2023-11-21|Electronics|     110|  13000|
|2023-11-22|Electronics|     105|  12500|
|2023-11-20|   Clothing|     300|  15000|
|2023-11-21|   Clothing|     280|  14000|
|2023-11-22|   Clothing|     320|  16000|
|2023-11-20|      Books|     150|   9000|
|2023-11-21|      Books|     200|  12000|
|2023-11-22|      Books|     180|  10000|
+----------+-----------+--------+-------+

+-----------+----------+----------+----------+
|   category|2023-11-20|2023-11-21|2023-11-22|
+-----------+----------+----------+----------+
|      Books|    9000.0|   12000.0|   10000.0|
|   Clothing|   15000.0|   14000.0|   16000.0|
|Electronics|   12000.0|   13000.0|   12500.0|
+-----------+----------+----------+----------+



In [21]:
# Завершение сеанса Spark
spark.stop()